In [ ]:
import cv2
import dlib
from scipy.spatial import distance
import imutils
from pygame import mixer
import winsound

In [ ]:
face_detector = dlib.get_frontal_face_detector()
landmark_predictor =dlib.shape_predictor(r"C:\Users\preanaisha\Downloads\archive\shape_predictor_68_face_landmarks.dat")


In [ ]:
def calculate_ear(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear



In [ ]:
def calculate_mar(mouth):
    A = distance.euclidean(mouth[2], mouth[10])
    B = distance.euclidean(mouth[4], mouth[8])
    C = distance.euclidean(mouth[0], mouth[6])
    mar = (A + B) / (2.0 * C)
    return mar

In [ ]:
def play_alarm():
    winsound.Beep(1000, 1000)  # Frequency = 1000 Hz, Duration = 1000 ms

In [ ]:
EAR_THRESHOLD = 0.25
FRAME_THRESHOLD = 20
MAR_THRESHOLD = 0.7

In [ ]:
cap = cv2.VideoCapture(0)
drowsy_frame_count = 0
yawning_frame_count = 0


In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray)
    
    for face in faces:
        landmarks = landmark_predictor(gray, face)
        left_eye = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)]
        right_eye = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)]
        mouth = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(48, 68)]
        
        left_ear = calculate_ear(left_eye)
        right_ear = calculate_ear(right_eye)
        avg_ear = (left_ear + right_ear) / 2.0
        mar = calculate_mar(mouth)
        
        if avg_ear < EAR_THRESHOLD:
            drowsy_frame_count += 1
            if drowsy_frame_count >= FRAME_THRESHOLD:
                cv2.putText(frame, "FATIGUE ALERT!", (50, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                play_alarm()
        else:
            drowsy_frame_count = 0
        
        if mar > MAR_THRESHOLD:
            yawning_frame_count += 1
            if yawning_frame_count >= FRAME_THRESHOLD:
                cv2.putText(frame, "YAWNING ALERT!", (50, 150), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
        else:
            yawning_frame_count = 0
       
    
    cv2.imshow("AUTOMATED FATIGUE DETECTION SYSTEM", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


In [ ]:
cap.release()
cv2.destroyAllWindows()
